In [ ]:
import pypsa
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import folium
import math
import networkx as nx

In [ ]:
network_nodes = pd.read_csv('database/plotting_generators_substation.csv', sep = ",", quotechar = "'")

network_nodes

In [ ]:
basemap = folium.Map(location = [-0.152375, 113.899132], zoom_start = 6)

maxcapacity = max(network_nodes.p_nom)

for i in range(len(network_nodes)):
    popupinfo = (network_nodes.node.iloc[i]+ ",p_nom:" + str(network_nodes.p_nom.iloc[i]))
    folium.CircleMarker(location = [network_nodes.lat.iloc[i], network_nodes.lon.iloc[i]], 
                       tooltip = popupinfo, color = "green", fill = True, 
                       radius = math.sqrt(50 * network_nodes.p_nom.iloc[i] / maxcapacity)).add_to(basemap)
    
basemap.save("initial_nodes.html")

display(basemap)

In [ ]:
network_edges = pd.read_csv('database/plotting_edges_generators_edit.csv', sep = ",", quotechar = "'")

network_edges

In [ ]:
basemap = folium.Map(location = [-0.152375, 113.899132], zoom_start = 6)

maxcapacity = max(network_nodes.p_nom)

for i in range(len(network_nodes)):
    popupinfo = (network_nodes.node.iloc[i]+ ",p_nom:" + str(network_nodes.p_nom.iloc[i]))
    folium.CircleMarker(location = [network_nodes.lat.iloc[i], network_nodes.lon.iloc[i]], 
                       tooltip = popupinfo, color = "green", fill = True, 
                       radius = math.sqrt(20 * network_nodes.p_nom.iloc[i] / maxcapacity)).add_to(basemap)

for i in range(len(network_edges)):
    points = [[network_edges.lat0.iloc[i], network_edges.lon0.iloc[i]],
             [network_edges.lat1.iloc[i], network_edges.lon1.iloc[i]]]
    folium.PolyLine(points, color = "yellow", opacitiy = 0.7).add_to(basemap)
    
basemap.save("initial_network.html")

display(basemap)

In [ ]:
nodes_network = pd.read_csv('database/GENERATORS_01.csv', sep = ",", quotechar = "'")

nodes_network

In [ ]:
network = pd.read_csv('database/plotting_edges_generators.csv')

network

In [ ]:
kalimantan_network = nx.from_pandas_edgelist(network, 'node0', 'node1', ['length_m'], create_using = nx.Graph)

In [ ]:
nx.draw_networkx(kalimantan_network, with_labels = False)

In [ ]:
print(len(kalimantan_network.nodes))
kalimantan_network.nodes()

In [ ]:
for index, row in network.iterrows():
    kalimantan_network.nodes[row['node0']]['coords'] = row['lon0'],row['lat0']
    kalimantan_network.nodes[row['node1']]['coords'] = row['lon1'],row['lat1']
    kalimantan_network.nodes[row['node0']]['type'] = row['type']
    kalimantan_network.nodes[row['node1']]['type'] = row['type']

In [ ]:
print(len(kalimantan_network.nodes))
print(kalimantan_network.nodes(data = True))

In [ ]:
nx.get_node_attributes(kalimantan_network, 'coords')

In [ ]:
nx.draw(kalimantan_network, nx.get_node_attributes(kalimantan_network, 'coords'), with_labels = False, node_size = 1)

In [ ]:
nx.get_node_attributes(kalimantan_network, 'type')

In [ ]:
color_type_map = {'substation': 'r', 'generator': 'b'}

In [ ]:
nx.draw(kalimantan_network, nx.get_node_attributes(kalimantan_network, 'coords'),
        node_color = [color_type_map[node[1]['type']]
                      for node in kalimantan_network.nodes(data=True)], with_labels = False, node_size = 10)

In [ ]:
def substation_nodes(kalimantan_network, type, substation):

    result = []

    d = nx.get_node_attributes(kalimantan_network, type)

    for key, v in d.items():
        if(v == substation):
            result.append(key)

    return result

In [ ]:
print(substation_nodes(kalimantan_network, 'type', 'substation'))

In [ ]:
from itertools import product

In [ ]:
def generator_nodes(kalimantan_network, type, generator):

    result = []

    d = nx.get_node_attributes(kalimantan_network, type)

    for key, v in d.items():
        if(v == generator):
            result.append(key)

    return result

In [ ]:
print(generator_nodes(kalimantan_network, 'type', 'generator'))

In [ ]:
nx.voronoi_cells(kalimantan_network, substation_nodes(kalimantan_network, 'type', 'substation'))